In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import distance
from multiprocessing import Pool
import json
%matplotlib inline
pd.set_option('display.max_columns', 100)

In [8]:
chunks = ['S'+str(i) for i in range(1,13)]
data = {}
clean_data = {}

for f in os.listdir('/home/katya/start/HIS3InterspeciesEpistasis/Data/'):
    if 'csv' in f:
        data[f[:-16]] = pd.DataFrame.from_csv('/home/katya/start/HIS3InterspeciesEpistasis/Data/' + f, sep = '\t')
        clean_data[f[:-16]] = data[f[:-16]][(data[f[:-16]].nonsense == 0) & (data[f[:-16]].middle == 1)]
        clean_data[f[:-16]] = clean_data[f[:-16]].sample(5000)

In [10]:
def write_json(chunk):
    print chunk
    
    fitness_threshold_1 = 0.8*np.mean(clean_data[chunk].s[clean_data[chunk].dist_Scer==0])
    fitness_threshold_2 = 0.6*np.mean(clean_data[chunk].s[clean_data[chunk].dist_Scer==0])
    fitness_threshold_3 = 0.4*np.mean(clean_data[chunk].s[clean_data[chunk].dist_Scer==0])
    fitness_threshold_4 = 0.2*np.mean(clean_data[chunk].s[clean_data[chunk].dist_Scer==0])
    
    sqs = list(clean_data[chunk].index)
    
    data = {}
    data['nodes'] = []
    counter = 0
    for i in range(len(sqs)):
        counter+=1
        data['nodes'].append({'name':'%s, distance: %d, fitness: %.2f' % (clean_data[chunk].mut_list[i], clean_data[chunk].dist_Scer[i], clean_data[chunk].s[i]),
                              'x':(clean_data[chunk].dist_Scer[i] + np.random.normal(0, .05)), 
                              'y':-(clean_data[chunk].s[i]*10)})
        
    data['connections'] = []
    
    done = []
    for i1 in range(len(sqs)):
        if i1%1000 == 0:
            print i1
        done.append(sqs[i1])
        for i2 in range(len(sqs)):
            if sqs[i2] not in done and clean_data[chunk].dist_Scer[i1]!=clean_data[chunk].dist_Scer[i2] \
            and distance.hamming(sqs[i1],sqs[i2]) == 1:
                
                if min(clean_data[chunk].s[i1], clean_data[chunk].s[i2])>=fitness_threshold_1:
                    data['connections'].append({'source':data['nodes'][i1]['name'], 'target':data['nodes'][i2]['name'], 'color':'#00AEE9'})
                
                elif min(clean_data[chunk].s[i1], clean_data[chunk].s[i2])>=fitness_threshold_2:
                    data['connections'].append({'source':data['nodes'][i1]['name'], 'target':data['nodes'][i2]['name'], 'color':'#34ADD3'})
                
                elif min(clean_data[chunk].s[i1], clean_data[chunk].s[i2])>=fitness_threshold_3:
                    data['connections'].append({'source':data['nodes'][i1]['name'], 'target':data['nodes'][i2]['name'], 'color':'#70ADC7'})
                
                elif min(clean_data[chunk].s[i1], clean_data[chunk].s[i2])>=fitness_threshold_4:
                    data['connections'].append({'source':data['nodes'][i1]['name'], 'target':data['nodes'][i2]['name'], 'color':'#9AABB4'})
                
                else:
                    data['connections'].append({'source':data['nodes'][i1]['name'], 'target':data['nodes'][i2]['name'], 'color':'#A7A9AC'})
                
    with open('/home/katya/start/HIS3InterspeciesEpistasis/Analysis/Katya/Fig2/' + chunk + '.json', 'w+') as outfile:  
        json.dump(data, outfile)

In [ ]:
pool = Pool()
pool.map(write_json, chunks)

S1
S6
S8
S11
S12
S7
S10
S9
S2
S5
S3
S4
0
0
0
0
0
0
0
0
0
0
0
0
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
3000
3000
3000
3000
3000
3000
3000
3000
4000
3000
3000
4000
4000
4000
3000
4000
4000
4000
3000
4000
4000
4000
4000
4000


In [12]:
x

NameError: name 'x' is not defined